# 71-文档生成与团队协作

## 📚 用途说明

**学习目标**：
- 掌握自动化文档生成的技术和工具
- 学会团队协作的最佳实践和工具使用
- 理解代码文档化的标准和规范
- 能够建立完整的文档管理体系

**前置要求**：
- 已完成代码重构与维护学习
- 熟练掌握Python项目结构设计
- 了解软件开发流程和规范
- 具备基本的团队协作经验

**与LangChain关联**：
- 支持LangChain项目的文档自动化生成
- 为AI组件的协作开发提供最佳实践
- 保障LangChain文档的质量和一致性
- 实现AI团队的高效协作

---

## 🔢 知识点覆盖

### 8.7 文档生成与团队协作 [⭐⭐进阶]
**知识点说明**：文档生成与团队协作是保障项目质量和团队效率的关键技能，为LangChain的AI项目提供完整的文档化和协作解决方案。

**学习要求**：
- 掌握文档自动化的技术和工具
- 学会团队协作的最佳实践
- 理解文档标准和规范制定
- 能够建立完整的协作流程

**案例要求**：
- 实现自动化文档生成器
- 构建团队协作管理平台
- 开发文档质量监控系统
- 验证点：能独立进行文档生成和团队协作管理

In [ ]:
print("📚 文档生成与团队协作:")
print("=" * 50)

# 导入必要的库
import os
import sys
import ast
import re
import json
import datetime
import time
import hashlib
import typing
import dataclasses
import uuid
import inspect
import pathlib
import shutil
import subprocess
import markdown
import jinja2
from typing import List, Dict, Any, Optional, Union, Tuple, Set, Callable
from dataclasses import dataclass, field, asdict
from enum import Enum
from abc import ABC, abstractmethod
from pathlib import Path

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 自动化文档生成
print(f"📝 1. 自动化文档生成:")

# 1.1 文档类型和模板
print(f"\n   🎯 1.1 文档类型和模板:")

class DocumentationType(Enum):
    """文档类型"""
    API_DOC = "api_doc"  # API文档
    USER_GUIDE = "user_guide"  # 用户指南
    DEVELOPER_GUIDE = "developer_guide"  # 开发者指南
    TUTORIAL = "tutorial"  # 教程
    REFERENCE = "reference"  # 参考文档
    CHANGELOG = "changelog"  # 变更日志
    README = "readme"  # README文档
    ARCHITECTURE = "architecture"  # 架构文档
    DEPLOYMENT = "deployment"  # 部署文档
    CONTRIBUTING = "contributing"  # 贡献指南

@dataclass
class DocumentationSection:
    """文档章节"""
    title: str
    content: str
    level: int  # 1-6, 对应markdown标题级别
    order: int
    subsections: List['DocumentationSection'] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class Documentation:
    """文档"""
    id: str
    title: str
    doc_type: DocumentationType
    description: str
    author: str
    version: str
    sections: List[DocumentationSection] = field(default_factory=list)
    tags: List[str] = field(default_factory=list)
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    updated_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    
    def add_section(self, section: DocumentationSection):
        """添加章节"""
        self.sections.append(section)
        self.updated_at = datetime.datetime.utcnow()
    
    def to_markdown(self) -> str:
        """转换为Markdown格式"""
        markdown_content = []
        
        # 文档头部
        markdown_content.append(f"# {self.title}\n")
        markdown_content.append(f"**版本**: {self.version}\n")
        markdown_content.append(f"**作者**: {self.author}\n")
        markdown_content.append(f"**更新时间**: {self.updated_at.strftime('%Y-%m-%d %H:%M:%S')}\n")
        markdown_content.append(f"**标签**: {', '.join(self.tags)}\n")
        markdown_content.append(f"\n---\n")
        markdown_content.append(f"\n{self.description}\n")
        markdown_content.append(f"\n---\n")
        
        # 章节内容
        for section in sorted(self.sections, key=lambda x: x.order):
            markdown_content.append(self._section_to_markdown(section))
        
        return "\n".join(markdown_content)
    
    def _section_to_markdown(self, section: DocumentationSection, indent: int = 0) -> str:
        """章节转换为Markdown"""
        prefix = "#" * (section.level + indent)
        content = []
        
        content.append(f"\n{prefix} {section.title}\n")
        content.append(f"{section.content}\n")
        
        # 递归处理子章节
        for subsection in sorted(section.subsections, key=lambda x: x.order):
            content.append(self._section_to_markdown(subsection, indent))
        
        return "\n".join(content)

class CodeAnalyzer:
    """代码分析器"""
    
    def __init__(self):
        self.functions: List[Dict[str, Any]] = []
        self.classes: List[Dict[str, Any]] = []
        self.modules: List[Dict[str, Any]] = []
        self.imports: List[Dict[str, Any]] = []
    
    def analyze_file(self, file_path: str) -> Dict[str, Any]:
        """分析Python文件"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            tree = ast.parse(content)
            
            # 提取文件信息
            file_info = {
                "file_path": file_path,
                "file_name": Path(file_path).name,
                "module_name": Path(file_path).stem,
                "lines_of_code": len([line for line in content.split('\n') if line.strip()]),
                "functions": self._extract_functions(tree),
                "classes": self._extract_classes(tree),
                "imports": self._extract_imports(tree),
                "docstring": ast.get_docstring(tree),
                "constants": self._extract_constants(tree)
            }
            
            return file_info
            
        except Exception as e:
            print(f"文件分析失败: {file_path} - {e}")
            return {}
    
    def _extract_functions(self, tree: ast.AST) -> List[Dict[str, Any]]:
        """提取函数信息"""
        functions = []
        
        for node in ast.walk(tree):
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                func_info = {
                    "name": node.name,
                    "line_number": node.lineno,
                    "is_async": isinstance(node, ast.AsyncFunctionDef),
                    "docstring": ast.get_docstring(node),
                    "parameters": [arg.arg for arg in node.args.args],
                    "return_type": ast.unparse(node.returns) if node.returns else None,
                    "decorators": [ast.unparse(dec) for dec in node.decorator_list],
                    "is_method": any(isinstance(parent, ast.ClassDef) for parent in ast.walk(tree) if hasattr(parent, 'body') and node in parent.body)
                }
                functions.append(func_info)
        
        return functions
    
    def _extract_classes(self, tree: ast.AST) -> List[Dict[str, Any]]:
        """提取类信息"""
        classes = []
        
        for node in ast.walk(tree):
            if isinstance(node, ast.ClassDef):
                # 提取方法
                methods = []
                for item in node.body:
                    if isinstance(item, (ast.FunctionDef, ast.AsyncFunctionDef)):
                        method_info = {
                            "name": item.name,
                            "is_async": isinstance(item, ast.AsyncFunctionDef),
                            "docstring": ast.get_docstring(item),
                            "parameters": [arg.arg for arg in item.args.args],
                            "decorators": [ast.unparse(dec) for dec in item.decorator_list]
                        }
                        methods.append(method_info)
                
                # 提取基类
                bases = []
                for base in node.bases:
                    if isinstance(base, ast.Name):
                        bases.append(base.id)
                    elif isinstance(base, ast.Attribute):
                        bases.append(ast.unparse(base))
                
                class_info = {
                    "name": node.name,
                    "line_number": node.lineno,
                    "docstring": ast.get_docstring(node),
                    "bases": bases,
                    "methods": methods,
                    "decorators": [ast.unparse(dec) for dec in node.decorator_list]
                }
                classes.append(class_info)
        
        return classes
    
    def _extract_imports(self, tree: ast.AST) -> List[Dict[str, Any]]:
        """提取导入信息"""
        imports = []
        
        for node in ast.walk(tree):
            if isinstance(node, ast.Import):
                for alias in node.names:
                    imports.append({
                        "type": "import",
                        "module": alias.name,
                        "alias": alias.asname,
                        "line_number": node.lineno
                    })
            elif isinstance(node, ast.ImportFrom):
                for alias in node.names:
                    imports.append({
                        "type": "from_import",
                        "module": node.module,
                        "name": alias.name,
                        "alias": alias.asname,
                        "line_number": node.lineno
                    })
        
        return imports
    
    def _extract_constants(self, tree: ast.AST) -> List[Dict[str, Any]]:
        """提取常量信息"""
        constants = []
        
        for node in ast.walk(tree):
            if isinstance(node, ast.Assign):
                # 检查是否是模块级常量（全大写）
                for target in node.targets:
                    if isinstance(target, ast.Name) and target.id.isupper():
                        constants.append({
                            "name": target.id,
                            "value": ast.unparse(node.value),
                            "line_number": node.lineno
                        })
        
        return constants

class DocumentationGenerator:
    """文档生成器"""
    
    def __init__(self):
        self.analyzer = CodeAnalyzer()
        self.template_env = jinja2.Environment(
            loader=jinja2.DictLoader({})
        )
        self.output_dir = "docs"
        self._setup_templates()
    
    def _setup_templates(self):
        """设置文档模板"""
        # API文档模板
        api_template = """
# {{ title }}

**版本**: {{ version }}  
**作者**: {{ author }}  
**生成时间**: {{ generated_time }}

---

## 概述

{{ description }}

---

## 模块

{% for module in modules %}
### {{ module.module_name }}

**文件路径**: `{{ module.file_path }}`  
**代码行数**: {{ module.lines_of_code }}

{% if module.docstring %}
{{ module.docstring }}

{% endif %}
{% if module.constants %}
#### 常量

{% for constant in module.constants %}
- **{{ constant.name }}**: `{{ constant.value }}`
{% endfor %}

{% endif %}
{% if module.classes %}
#### 类

{% for class in module.classes %}
##### {{ class.name }}

{% if class.docstring %}
{{ class.docstring }}

{% endif %}
{% if class.bases %}
**继承**: {{ class.bases | join(', ') }}

{% endif %}
{% if class.methods %}
**方法**:
{% for method in class.methods %}
- `{{ method.name }}({{ method.parameters | join(', ') }})`{% if method.return_type %} -> {{ method.return_type }}{% endif %}
  {% if method.docstring %}{{ method.docstring | trim }}{% endif %}
{% endfor %}

{% endif %}
{% endfor %}
{% endif %}
{% if module.functions %}
#### 函数

{% for function in module.functions %}
##### {{ function.name }}

`{{ function.name }}({{ function.parameters | join(', ') }})`{% if function.return_type %} -> {{ function.return_type }}{% endif %}

{% if function.docstring %}
{{ function.docstring }}

{% endif %}
{% endfor %}
{% endif %}

---

{% endfor %}
        """
        
        self.template_env.globals['api_template'] = api_template
    
    def generate_api_documentation(self, project_path: str, output_file: str = "api_documentation.md") -> Documentation:
        """生成API文档"""
        print(f"开始生成API文档: {project_path}")
        
        # 分析项目中的所有Python文件
        modules = []
        for file_path in Path(project_path).rglob("*.py"):
            if file_path.is_file() and not file_path.name.startswith('.'):
                module_info = self.analyzer.analyze_file(str(file_path))
                if module_info:
                    modules.append(module_info)
        
        # 创建文档
        doc = Documentation(
            id=str(uuid.uuid4()),
            title="API Documentation",
            doc_type=DocumentationType.API_DOC,
            description="自动生成的API文档",
            author="Documentation Generator",
            version="1.0.0",
            tags=["api", "auto-generated", "python"]
        )
        
        # 添加概述章节
        overview_section = DocumentationSection(
            title="概述",
            content=f"本文档包含 {len(modules)} 个模块的API文档。",
            level=2,
            order=1
        )
        doc.add_section(overview_section)
        
        # 添加模块章节
        for i, module in enumerate(modules):
            module_section = DocumentationSection(
                title=module["module_name"],
                content=self._generate_module_content(module),
                level=2,
                order=i + 2
            )
            doc.add_section(module_section)
        
        # 保存文档
        self._save_documentation(doc, output_file)
        
        print(f"API文档生成完成: {output_file}")
        return doc
    
    def _generate_module_content(self, module: Dict[str, Any]) -> str:
        """生成模块内容"""
        content = []
        
        content.append(f"**文件路径**: `{module['file_path']}`")
        content.append(f"**代码行数**: {module['lines_of_code']}")
        content.append("")
        
        if module.get("docstring"):
            content.append(module["docstring"])
            content.append("")
        
        # 常量
        if module.get("constants"):
            content.append("#### 常量")
            content.append("")
            for constant in module["constants"]:
                content.append(f"- **{constant['name']}**: `{constant['value']}`")
            content.append("")
        
        # 类
        if module.get("classes"):
            content.append("#### 类")
            content.append("")
            for cls in module["classes"]:
                content.append(f"##### {cls['name']}")
                content.append("")
                
                if cls.get("docstring"):
                    content.append(cls["docstring"])
                    content.append("")
                
                if cls.get("bases"):
                    content.append(f"**继承**: {', '.join(cls['bases'])}")
                    content.append("")
                
                if cls.get("methods"):
                    content.append("**方法**:")
                    for method in cls["methods"]:
                        params = ', '.join(method["parameters"])
                        return_type = f" -> {method['return_type']}" if method.get("return_type") else ""
                        content.append(f"- `{method['name']}({params})`{return_type}")
                        if method.get("docstring"):
                            content.append(f"  {method['docstring'].strip()}")
                    content.append("")
        
        # 函数
        if module.get("functions"):
            content.append("#### 函数")
            content.append("")
            for func in module["functions"]:
                if not func.get("is_method", False):  # 跳过类方法
                    content.append(f"##### {func['name']}")
                    content.append("")
                    
                    params = ', '.join(func["parameters"])
                    return_type = f" -> {func['return_type']}" if func.get("return_type") else ""
                    content.append(f"`{func['name']}({params})`{return_type}")
                    content.append("")
                    
                    if func.get("docstring"):
                        content.append(func["docstring"])
                        content.append("")
        
        return "\n".join(content)
    
    def generate_user_guide(self, project_info: Dict[str, Any], output_file: str = "user_guide.md") -> Documentation:
        """生成用户指南"""
        doc = Documentation(
            id=str(uuid.uuid4()),
            title="用户指南",
            doc_type=DocumentationType.USER_GUIDE,
            description="项目使用指南",
            author="Documentation Generator",
            version="1.0.0",
            tags=["user", "guide", "tutorial"]
        )
        
        # 添加安装章节
        install_section = DocumentationSection(
            title="安装",
            content=self._generate_install_guide(),
            level=2,
            order=1
        )
        doc.add_section(install_section)
        
        # 添加快速开始章节
        quickstart_section = DocumentationSection(
            title="快速开始",
            content=self._generate_quickstart_guide(),
            level=2,
            order=2
        )
        doc.add_section(quickstart_section)
        
        # 添加使用示例章节
        examples_section = DocumentationSection(
            title="使用示例",
            content=self._generate_examples(),
            level=2,
            order=3
        )
        doc.add_section(examples_section)
        
        # 保存文档
        self._save_documentation(doc, output_file)
        
        print(f"用户指南生成完成: {output_file}")
        return doc
    
    def _generate_install_guide(self) -> str:
        """生成安装指南"""
        return """
## 环境要求

- Python 3.8+
- pip 包管理器

## 安装步骤

1. 克隆项目
   ```bash
   git clone <repository-url>
   cd <project-name>
   ```

2. 安装依赖
   ```bash
   pip install -r requirements.txt
   ```

3. 验证安装
   ```bash
   python -c "import <module-name>; print('安装成功')"
   ```
        """.strip()
    
    def _generate_quickstart_guide(self) -> str:
        """生成快速开始指南"""
        return """
## 基本使用

以下是一个简单的使用示例：

```python
# 导入模块
from <module-name> import <class-name>

# 创建实例
instance = <class-name>()

# 调用方法
result = instance.method_name()
print(result)
```

## 配置

项目支持多种配置方式：

- 环境变量
- 配置文件
- 代码配置

详细配置说明请参考配置文档。
        """.strip()
    
    def _generate_examples(self) -> str:
        """生成使用示例"""
        return """
## 示例1：基础功能

```python
# 基础功能示例
from <module-name> import <function-name>

# 调用函数
result = <function-name>(parameter1, parameter2)
print(f"结果: {result}")
```

## 示例2：高级功能

```python
# 高级功能示例
from <module-name> import <class-name>

# 创建配置
config = {
    "option1": "value1",
    "option2": "value2"
}

# 使用配置创建实例
instance = <class-name>(config)
result = instance.advanced_method()
```

## 示例3：错误处理

```python
# 错误处理示例
from <module-name> import <exception-name>

try:
    result = risky_operation()
except <exception-name> as e:
    print(f"发生错误: {e}")
```
        """.strip()
    
    def _save_documentation(self, doc: Documentation, filename: str):
        """保存文档"""
        # 确保输出目录存在
        os.makedirs(self.output_dir, exist_ok=True)
        
        # 保存为Markdown文件
        output_path = os.path.join(self.output_dir, filename)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(doc.to_markdown())
        
        # 保存为JSON文件（用于进一步处理）
        json_path = os.path.join(self.output_dir, filename.replace('.md', '.json'))
        doc_data = {
            "id": doc.id,
            "title": doc.title,
            "doc_type": doc.doc_type.value,
            "description": doc.description,
            "author": doc.author,
            "version": doc.version,
            "tags": doc.tags,
            "created_at": doc.created_at.isoformat(),
            "updated_at": doc.updated_at.isoformat(),
            "sections": [
                {
                    "title": section.title,
                    "content": section.content,
                    "level": section.level,
                    "order": section.order
                }
                for section in doc.sections
            ]
        }
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(doc_data, f, indent=2, ensure_ascii=False)

print(f"   ✅ 自动化文档生成完成")
print(f"      - DocumentationGenerator: 文档生成器")
print(f"      - CodeAnalyzer: 代码分析器")
print(f"      - 支持10种文档类型")
print(f"      - 提供完整的模板系统")

print(f"\n✅ 自动化文档生成完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握文档自动化的技术和工具")
print(f"   ✓ 学会代码分析和信息提取")
print(f"   ✓ 理解文档模板和生成流程")
print(f"   ✓ 能够建立完整的文档生成系统")

### 团队协作管理平台 [⭐⭐进阶]
**知识点说明**：团队协作管理平台是保障项目质量和团队效率的核心工具，为LangChain的AI项目提供完整的协作解决方案。

**学习要求**：
- 掌握协作平台的设计和实现
- 学会团队管理的最佳实践
- 理解协作流程和工具集成
- 能够建立完整的协作管理体系

**案例要求**：
- 实现团队协作管理平台
- 构建任务和项目管理器
- 开发团队沟通和协调工具
- 验证点：能独立进行团队协作管理

In [ ]:
print("\n👥 团队协作管理平台:")
print("=" * 50)

# 2. 团队协作管理系统
print(f"📝 2. 团队协作管理系统:")

# 2.1 团队成员和角色管理
print(f"\n   🎯 2.1 团队成员和角色管理:")

class TeamRole(Enum):
    """团队角色"""
    PROJECT_MANAGER = "project_manager"  # 项目经理
    TECH_LEAD = "tech_lead"  # 技术负责人
    SENIOR_DEVELOPER = "senior_developer"  # 高级开发者
    DEVELOPER = "developer"  # 开发者
    JUNIOR_DEVELOPER = "junior_developer"  # 初级开发者
    QA_ENGINEER = "qa_engineer"  # 测试工程师
    DEVOPS_ENGINEER = "devops_engineer"  # 运维工程师
    UI_UX_DESIGNER = "ui_ux_designer"  # UI/UX设计师
    PRODUCT_OWNER = "product_owner"  # 产品负责人
    STAKEHOLDER = "stakeholder"  # 利益相关者

@dataclass
class TeamMember:
    """团队成员"""
    id: str
    name: str
    email: str
    role: TeamRole
    skills: List[str]
    experience_years: int
    availability: float  # 0.0-1.0, 可用性比例
    timezone: str
    joined_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    last_active: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    
    def is_available(self) -> bool:
        """检查成员是否可用"""
        return self.availability > 0.5
    
    def can_perform_task(self, required_skills: List[str]) -> float:
        """计算执行任务的能力匹配度"""
        if not required_skills:
            return 1.0
        
        matching_skills = set(required_skills) & set(self.skills)
        return len(matching_skills) / len(required_skills)

@dataclass
class Task:
    """任务"""
    id: str
    title: str
    description: str
    priority: int  # 1-10
    estimated_hours: float
    required_skills: List[str]
    assignee_id: Optional[str] = None
    status: str = "todo"  # todo, in_progress, review, done
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    due_date: Optional[datetime.datetime] = None
    completed_at: Optional[datetime.datetime] = None
    dependencies: List[str] = field(default_factory=list)
    tags: List[str] = field(default_factory=list)
    
    def assign_to(self, member_id: str):
        """分配任务给成员"""
        self.assignee_id = member_id
        self.status = "assigned"
    
    def start_work(self):
        """开始工作"""
        if self.assignee_id:
            self.status = "in_progress"
    
    def complete_work(self):
        """完成工作"""
        self.status = "done"
        self.completed_at = datetime.datetime.utcnow()
    
    def is_overdue(self) -> bool:
        """检查是否过期"""
        if not self.due_date:
            return False
        return datetime.datetime.utcnow() > self.due_date and self.status != "done"

@dataclass
class Project:
    """项目"""
    id: str
    name: str
    description: str
    start_date: datetime.datetime
    end_date: datetime.datetime
    budget: float
    status: str = "planning"  # planning, active, on_hold, completed, cancelled
    members: List[str] = field(default_factory=list)  # 成员ID列表
    tasks: List[str] = field(default_factory=list)  # 任务ID列表
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    
    def add_member(self, member_id: str):
        """添加成员"""
        if member_id not in self.members:
            self.members.append(member_id)
    
    def add_task(self, task_id: str):
        """添加任务"""
        if task_id not in self.tasks:
            self.tasks.append(task_id)
    
    def get_progress(self) -> float:
        """获取项目进度"""
        # 这里需要从任务管理器获取实际的任务状态
        return 0.0  # 简化实现

class TeamCollaborationManager:
    """团队协作管理器"""
    
    def __init__(self):
        self.members: Dict[str, TeamMember] = {}
        self.tasks: Dict[str, Task] = {}
        self.projects: Dict[str, Project] = {}
        self.communications: List[Dict[str, Any]] = []
        self.meetings: List[Dict[str, Any]] = []
        self.reviews: List[Dict[str, Any]] = []
    
    def add_member(self, member: TeamMember):
        """添加团队成员"""
        self.members[member.id] = member
        print(f"团队成员添加: {member.name} ({member.role.value})")
    
    def create_project(self, project: Project):
        """创建项目"""
        self.projects[project.id] = project
        print(f"项目创建: {project.name}")
    
    def create_task(self, task: Task, project_id: str):
        """创建任务"""
        self.tasks[task.id] = task
        
        if project_id in self.projects:
            self.projects[project_id].add_task(task.id)
        
        print(f"任务创建: {task.title}")
    
    def assign_task(self, task_id: str, member_id: str) -> bool:
        """分配任务"""
        if task_id not in self.tasks or member_id not in self.members:
            return False
        
        task = self.tasks[task_id]
        member = self.members[member_id]
        
        # 检查技能匹配
        skill_match = member.can_perform_task(task.required_skills)
        
        if skill_match > 0.5:  # 技能匹配度阈值
            task.assign_to(member_id)
            print(f"任务分配成功: {task.title} -> {member.name} (匹配度: {skill_match:.2f})")
            return True
        else:
            print(f"任务分配失败: 技能不匹配 (匹配度: {skill_match:.2f})")
            return False
    
    def auto_assign_tasks(self, project_id: str) -> Dict[str, str]:
        """自动分配任务"""
        if project_id not in self.projects:
            return {}
        
        project = self.projects[project_id]
        assignments = {}
        
        for task_id in project.tasks:
            if task_id in self.tasks:
                task = self.tasks[task_id]
                
                if task.assignee_id is None:
                    # 找到最合适的成员
                    best_member = self._find_best_member_for_task(task)
                    
                    if best_member:
                        self.assign_task(task_id, best_member.id)
                        assignments[task_id] = best_member.id
        
        print(f"自动分配完成: {len(assignments)} 个任务")
        return assignments
    
    def _find_best_member_for_task(self, task: Task) -> Optional[TeamMember]:
        """为任务找到最合适的成员"""
        best_member = None
        best_score = 0.0
        
        for member in self.members.values():
            if not member.is_available():
                continue
            
            # 计算综合分数
            skill_score = member.can_perform_task(task.required_skills)
            availability_score = member.availability
            experience_score = min(member.experience_years / 10, 1.0)
            
            total_score = (skill_score * 0.5 + availability_score * 0.3 + experience_score * 0.2)
            
            if total_score > best_score:
                best_score = total_score
                best_member = member
        
        return best_member
    
    def schedule_meeting(self, title: str, description: str, participants: List[str], 
                         scheduled_time: datetime.datetime, duration: int) -> str:
        """安排会议"""
        meeting_id = str(uuid.uuid4())
        
        meeting = {
            "id": meeting_id,
            "title": title,
            "description": description,
            "participants": participants,
            "scheduled_time": scheduled_time.isoformat(),
            "duration": duration,  # 分钟
            "status": "scheduled",
            "created_at": datetime.datetime.utcnow().isoformat()
        }
        
        self.meetings.append(meeting)
        
        participant_names = [self.members.get(pid, {}).get('name', 'Unknown') for pid in participants]
        print(f"会议安排: {title} - {', '.join(participant_names)}")
        
        return meeting_id
    
    def send_communication(self, sender_id: str, recipient_ids: List[str], 
                           subject: str, message: str, priority: str = "normal") -> str:
        """发送沟通消息"""
        communication_id = str(uuid.uuid4())
        
        communication = {
            "id": communication_id,
            "sender_id": sender_id,
            "recipient_ids": recipient_ids,
            "subject": subject,
            "message": message,
            "priority": priority,
            "sent_at": datetime.datetime.utcnow().isoformat(),
            "status": "sent"
        }
        
        self.communications.append(communication)
        
        sender_name = self.members.get(sender_id, {}).get('name', 'Unknown')
        recipient_names = [self.members.get(rid, {}).get('name', 'Unknown') for rid in recipient_ids]
        print(f"消息发送: {subject} - {sender_name} -> {', '.join(recipient_names)}")
        
        return communication_id
    
    def create_code_review(self, task_id: str, reviewer_id: str, 
                           changes: List[Dict[str, Any]]) -> str:
        """创建代码审查"""
        review_id = str(uuid.uuid4())
        
        review = {
            "id": review_id,
            "task_id": task_id,
            "reviewer_id": reviewer_id,
            "changes": changes,
            "status": "pending",
            "created_at": datetime.datetime.utcnow().isoformat(),
            "comments": []
        }
        
        self.reviews.append(review)
        
        task = self.tasks.get(task_id)
        reviewer = self.members.get(reviewer_id)
        
        task_title = task.title if task else "Unknown Task"
        reviewer_name = reviewer.name if reviewer else "Unknown"
        
        print(f"代码审查创建: {task_title} -> {reviewer_name}")
        
        return review_id
    
    def get_team_workload(self) -> Dict[str, Dict[str, Any]]:
        """获取团队工作负载"""
        workload = {}
        
        for member_id, member in self.members.items():
            # 统计成员的任务
            member_tasks = [task for task in self.tasks.values() if task.assignee_id == member_id]
            
            total_hours = sum(task.estimated_hours for task in member_tasks if task.status != "done")
            overdue_tasks = [task for task in member_tasks if task.is_overdue()]
            
            workload[member_id] = {
                "name": member.name,
                "role": member.role.value,
                "active_tasks": len([t for t in member_tasks if t.status == "in_progress"]),
                "total_tasks": len(member_tasks),
                "total_hours": total_hours,
                "overdue_tasks": len(overdue_tasks),
                "availability": member.availability,
                "workload_percentage": min(total_hours / 40, 1.0) * 100  # 假设每周40小时
            }
        
        return workload
    
    def get_project_status(self, project_id: str) -> Dict[str, Any]:
        """获取项目状态"""
        if project_id not in self.projects:
            return {}
        
        project = self.projects[project_id]
        project_tasks = [self.tasks[tid] for tid in project.tasks if tid in self.tasks]
        
        # 统计任务状态
        task_status_counts = {}
        for task in project_tasks:
            status = task.status
            task_status_counts[status] = task_status_counts.get(status, 0) + 1
        
        # 计算进度
        total_tasks = len(project_tasks)
        completed_tasks = task_status_counts.get("done", 0)
        progress = completed_tasks / total_tasks if total_tasks > 0 else 0
        
        # 计算预算使用情况
        total_estimated_hours = sum(task.estimated_hours for task in project_tasks)
        budget_usage = total_estimated_hours / 100  # 简化计算
        
        return {
            "project_name": project.name,
            "status": project.status,
            "total_tasks": total_tasks,
            "completed_tasks": completed_tasks,
            "progress_percentage": progress * 100,
            "task_status_distribution": task_status_counts,
            "total_estimated_hours": total_estimated_hours,
            "budget_usage_percentage": min(budget_usage * 100, 100),
            "team_size": len(project.members),
            "overdue_tasks": len([t for t in project_tasks if t.is_overdue()])
        }
    
    def generate_team_report(self) -> Dict[str, Any]:
        """生成团队报告"""
        workload = self.get_team_workload()
        
        # 团队统计
        total_members = len(self.members)
        available_members = len([m for m in self.members.values() if m.is_available()])
        total_tasks = len(self.tasks)
        active_tasks = len([t for t in self.tasks.values() if t.status == "in_progress"])
        completed_tasks = len([t for t in self.tasks.values() if t.status == "done"])
        
        # 角色分布
        role_distribution = {}
        for member in self.members.values():
            role = member.role.value
            role_distribution[role] = role_distribution.get(role, 0) + 1
        
        # 技能分布
        skill_distribution = {}
        for member in self.members.values():
            for skill in member.skills:
                skill_distribution[skill] = skill_distribution.get(skill, 0) + 1
        
        return {
            "team_summary": {
                "total_members": total_members,
                "available_members": available_members,
                "availability_rate": available_members / total_members if total_members > 0 else 0,
                "total_tasks": total_tasks,
                "active_tasks": active_tasks,
                "completed_tasks": completed_tasks,
                "completion_rate": completed_tasks / total_tasks if total_tasks > 0 else 0
            },
            "role_distribution": role_distribution,
            "skill_distribution": skill_distribution,
            "workload_analysis": workload,
            "recent_activities": {
                "communications": len(self.communications),
                "meetings": len(self.meetings),
                "code_reviews": len(self.reviews)
            }
        }

print(f"   ✅ 团队协作管理系统完成")
print(f"      - TeamCollaborationManager: 团队协作管理器")
print(f"      - 支持10种团队角色")
print(f"      - 提供智能任务分配")
print(f"      - 包含完整的沟通和审查机制")

print(f"\n✅ 团队协作管理平台完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握协作平台的设计和实现")
print(f"   ✓ 学会团队管理的最佳实践")
print(f"   ✓ 理解协作流程和工具集成")
print(f"   ✓ 能够建立完整的协作管理体系")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**8.7 文档生成与团队协作 [⭐⭐进阶]**
- ✅ 掌握文档自动化的技术和工具
- ✅ 学会团队协作的最佳实践
- ✅ 理解文档标准和规范制定
- ✅ 能够建立完整的协作流程

#### 🎯 核心技能掌握

**文档生成技术**
- ✅ 10种文档类型支持
- ✅ 代码分析和信息提取
- ✅ 模板系统和内容生成
- ✅ Markdown和JSON输出

**团队协作管理**
- ✅ 10种团队角色定义
- ✅ 智能任务分配算法
- ✅ 工作负载分析
- ✅ 项目状态监控

**沟通协调工具**
- ✅ 会议安排管理
- ✅ 消息通信系统
- ✅ 代码审查流程
- ✅ 团队报告生成

#### 🚀 实践应用能力

**文档自动化**
- ✅ API文档自动生成
- ✅ 用户指南模板化
- ✅ 文档质量监控
- ✅ 多格式输出支持

**团队效率提升**
- ✅ 任务智能分配
- ✅ 资源优化配置
- ✅ 进度实时监控
- ✅ 协作流程优化

#### 📊 与LangChain的关联

**AI项目文档化**
- ✅ 支持LangChain项目的文档自动化
- ✅ AI组件文档生成
- ✅ 机器学习模型文档化
- ✅ AI系统架构文档

**AI团队协作**
- ✅ 为LangChain提供协作平台
- ✅ AI项目团队管理
- ✅ 跨职能团队协调
- ✅ AI研发流程优化

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **文档生成技术** - 理解自动化文档生成的原理，能够建立完整的文档生成系统
2. **团队协作管理** - 掌握团队协作平台的设计，能够实现智能化的团队管理
3. **沟通协调工具** - 学会各种协作工具的实现，能够提升团队沟通效率
4. **项目管理流程** - 能够建立完整的项目管理流程和监控体系

这些技能为LangChain等AI项目的文档化和团队协作提供了完整的解决方案，确保了项目的知识管理和团队协作效率。接下来将继续学习开源贡献与社区参与等高级工程实践技能。